In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import copy

In [2]:
data_dir = "/Users/ShreyaSethi/Desktop/Python/Projects/medical-assistant-chatbot/data/images"

# Define image transformations for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),   # Data augmentation for training
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # Imagenet mean
                             [0.229, 0.224, 0.225])  # Imagenet std
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

# Create DataLoader objects to iterate over datasets in batches
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32,
                             shuffle=True, num_workers=4)
               for x in ['train', 'val']}

# Sizes of datasets
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# Class names (e.g., ['Normal', 'Pneumonia'])
class_names = image_datasets['train'].classes

In [4]:
print("Classes:", class_names)
print("Train dataset size:", dataset_sizes['train'])
print("Validation dataset size:", dataset_sizes['val'])

Classes: ['NORMAL', 'PNEUMONIA']
Train dataset size: 5216
Validation dataset size: 16


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Replace the final fully connected layer to match number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Scheduler to decrease learning rate by factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Using device: cpu


/Users/ShreyaSethi/Desktop/Python/Projects/medical-assistant-chatbot/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ShreyaSethi/Desktop/Python/Projects/medical-assistant-chatbot/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 20)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Deep copy the best model weights
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val accuracy: {best_acc:.4f}")

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
trained_model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=15)

Epoch 1/15
--------------------


In [ ]:
# Set model to eval mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in dataloaders['val']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

In [ ]:
torch.save(model.state_dict(), 'pneumonia_model.pth')
print("Model saved as pneumonia_model.pth")

### INFERENCE

In [ ]:
import random
from PIL import Image

data_dir = "/Users/ShreyaSethi/Desktop/Python/Projects/medical-assistant-chatbot/data/images"
test_root = os.path.join(data_dir, "test")

image_paths = []
for root, dirs, files in os.walk(test_root):
    print(f"Scanning {root}...")
    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            full_path = os.path.join(root, file)
            print(f"Found image: {full_path}")
            image_paths.append(full_path)



results = []

# Inference loop with result collection
for img_path in image_paths:
    true_label = os.path.basename(os.path.dirname(img_path))
    image = Image.open(img_path).convert('RGB')
    transform = data_transforms['val']
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]

    results.append({
        "filename": os.path.basename(img_path),
        "true_label": true_label,
        "predicted_label": predicted_class
    })

# 1. Save to CSV
df = pd.DataFrame(results)
df.to_csv("test_predictions.csv", index=False)
print("Saved predictions to test_predictions.csv")

# 2. Compute Accuracy
correct = sum([row["true_label"] == row["predicted_label"] for row in results])
total = len(results)
accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f} ({correct}/{total})")

# 3. Visualize sample predictions
print("\n🎨 Visualizing 6 predictions:")
sample = random.sample(results, min(6, total))

plt.figure(figsize=(15, 8))
for i, row in enumerate(sample):
    img = Image.open(os.path.join(test_root, row["true_label"], row["filename"])).convert("RGB")
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.axis('off')
    color = "green" if row["true_label"] == row["predicted_label"] else "red"
    plt.title(f"True: {row['true_label']}\nPred: {row['predicted_label']}", color=color)

plt.tight_layout()
plt.show()